In [3]:
import openml
from mlaut.data import Data
import pandas as pd
from sklearn import preprocessing
import os

/media/viktor/Data/PhD/mlaut/py36/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
apikey = 'd2b1d13981d4abfb22895337baca924c'
openml.config.apikey = apikey
openml.config.set_cache_directory(os.path.expanduser('~/.openml/cache'))

In [5]:
classification_tasks = openml.tasks.list_tasks(task_type_id=1)
regression_tasks = openml.tasks.list_tasks(task_type_id=2)

In [6]:
tasks = openml.tasks.list_tasks(task_type_id=1)
#filter = tasks.task_type == 'Supervised Classification'

In [8]:
for t in tasks:
    print(t)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
18
20
22
23
24
25
26
27
28
29
30
31
32
34
35
36
37
38
39
40
41
42
43
45
47
48
49
50
51
52
53
54
55
57
58
59
60
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
201
202
203
204
205
206
207
208
209
210
211
212
213
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
248
250
252
253
254
255
256
257
258
259
260
261
262
264
265
266
267
268
269
270
271
272
273
275
277
278
279
280
281
282
283
284
285
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
1766
1767
1768
1769
1770
1771
1772
1773
1774
1775
1776
1777
1778
1779
1780
1782
1784
1786
1787
1788
1789
1790
1791
1792
1793
1794
1795
1796
1798
1799
1800
1801
1802
1803
1804
1805
1806
1807
1809
1811


145853
145854
145855
145856
145857
145858
145859
145860
145861
145862
145863
145864
145865
145866
145867
145868
145869
145870
145871
145872
145874
145875
145876
145877
145878
145879
145880
145881
145882
145883
145884
145885
145886
145887
145888
145889
145890
145891
145892
145893
145894
145895
145896
145897
145898
145899
145900
145901
145902
145903
145904
145905
145906
145907
145908
145909
145910
145911
145912
145913
145914
145915
145916
145917
145918
145919
145920
145921
145922
145923
145924
145925
145926
145927
145928
145929
145930
145931
145932
145933
145934
145935
145936
145937
145938
145939
145940
145941
145942
145943
145944
145945
145952
145953
145954
145955
145956
145957
145958
145959
145960
145961
145962
145963
145964
145966
145967
145968
145969
145970
145971
145972
145973
145974
145975
145976
145977
145978
145979
145982
145983
145984
145985
145986
145987
145988
145989
145990
145991
145992
145993
145994
145995
145996
145997
145998
145999
146000
146001
146002
146003
146004
146005

In [22]:
openml.datasets.get_dataset(238)

OpenMLServerException: Unknown dataset

In [17]:
dataset = openml.datasets.get_dataset(name='liver-disorders')
#missing_vals =dataset.__dict__['qualities']['NumberOfMissingValues']

TypeError: get_dataset() got an unexpected keyword argument 'name'

In [ ]:
X, names = dataset.get_data(return_attribute_names=True)

In [ ]:
data = Data()
input_io = data.open_hdf5('data/openml.h5', mode='a')

In [ ]:
for id in classification_tasks.keys():
    try:
        dataset = openml.datasets.get_dataset(id)
        X, names = dataset.get_data(return_attribute_names=True)

        #ignore datasets with empty values 
        num_missing_values = float(dataset.__dict__['qualities']['NumberOfMissingValues'])
        if num_missing_values > 0:
            print(f'skipping dataset {id}. Missing values')
            continue

        #ignore too big datasets
        if classification_tasks[id]['NumberOfInstances'] > 10000:
            print('skipping dataset {id}. It is too big')
            continue

        metadata = {
            'class_name': dataset.__dict__['default_target_attribute'],
            'source': 'OpenML',
            'dataset_name':dataset.__dict__['name']
        }
        class_name_index = names.index(metadata['class_name'])

        #Normalize the data
        scaler = preprocessing.StandardScaler(copy=True, with_mean=True, with_std=True)
        scaler.fit(X)
        x_transformed  = scaler.transform(X)
        x_transformed[:,class_name_index] = X[:, class_name_index]

        #Convert to DataFrame
        result = pd.DataFrame(x_transformed)
        result.columns=names
        result[metadata['class_name']] =  result[metadata['class_name']].astype(int)

        #save to hdf5
        input_io.save_pandas_dataset(dataset=result, save_loc='/openml', metadata=metadata)
        print(f'dataset {id} saved.')

    except:
        print(f'Cannot save dataset {id}')